In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from unidecode import unidecode
import plotly.graph_objects as go

%matplotlib inline

# Candidatos a vereadores por partido

In [168]:
df = pd.read_csv('input/tse/vereadores_total/2020.csv').drop(['Unnamed: 0'],axis=1)

In [170]:
df.sigla_partido.replace('PRN','PTC', inplace=True)
df.sigla_partido.replace('PPB','PP', inplace=True)
df.sigla_partido.replace('PFL','DEM', inplace=True)
df.sigla_partido.replace('PTN','PODE', inplace=True)
df.sigla_partido.replace('PT DO B','AVT', inplace=True)
df.sigla_partido.replace('PEN','PATRI', inplace=True)
df.sigla_partido.replace('PMDB','MDB', inplace=True)
df.sigla_partido.replace('PSDC','DC', inplace=True)
df.sigla_partido.replace('PPS','CDD', inplace=True)
df.sigla_partido.replace('PR','PL', inplace=True)

df.sigla_partido.replace('REPUBLICANOS','PRB', inplace=True)
df.sigla_partido.replace('AVANTE','AVT', inplace=True)
df.sigla_partido.replace('CIDADANIA','CDD', inplace=True)
df.sigla_partido.replace('SOLIDARIEDADE','SD', inplace=True)
df.sigla_partido.replace('PATRIOTA','PATRI', inplace=True)

# Mapa ideológico dos partidos

In [171]:
mapa_ideologico = {'PCB':'Esquerda',
                    'PT':'Esquerda',
                    'UP':'Esquerda',
                    'PSOL':'Esquerda',
                    'PSTU':'Esquerda',
                    'PC DO B':'Esquerda',
                    'PCO':'Esquerda',
                    'PDT':'Esquerda',
                    'REDE':'Esquerda',
                    'PSB':'Esquerda',
                    'PV':'Esquerda',
                    'SD':'Centro',
                    'PROS':'Centro',
                    'PTB':'Centro',
                    'PRB':'Centro',
                    'PP':'Centro',
                    'PMN':'Centro',
                    'PMB':'Centro',
                    'MDB':'Centro',
                    'PSDB':'Centro',
                    'CDD':'Centro',
                    'PSD':'Centro',
                    'DEM':'Direita',
                    'PL':'Direita',
                    'DC':'Direita',
                    'AVT':'Direita',
                    'PSC':'Direita',
                    'PODE':'Direita',
                    'PATRI':'Direita',
                    'NOVO':'Direita',
                    'PRTB':'Direita',
                    'PSL':'Direita',
                    'PTC':'Direita',
                    'PGT':'Esquerda',
                    'PST':'Esquerda',
                    'PHS':'Direita',
                    'PRP':'Direita',
                    'PPL':'Esquerda',
                    'PAN':'Centro',
                    'PRONA':'Direita'}

# Regiões e capitais

In [172]:
sigla_estado = {'AC':'Acre',
                'AL':'Alagoas',
                'AP':'Amapa',
                'AM':'Amazonas',
                'BA':'Bahia',
                'CE':'Ceara',
                'DF':'Distrito Federal',
                'ES':'Espirito Santo',
                'GO':'Goias',
                'MA':'Maranhao',
                'MT':'Mato Grosso',
                'MS':'Mato Grosso do Sul',
                'MG':'Minas Gerais',
                'PA':'Para',
                'PB':'Paraiba',
                'PR':'Parana',
                'PE':'Pernambuco',
                'PI':'Piaui',
                'RJ':'Rio de Janeiro',
                'RN':'Rio Grande do Norte',
                'RS':'Rio Grande do Sul',
                'RO':'Rondonia',
                'RR':'Roraima',
                'SC':'Santa Catarina',
                'SP':'Sao Paulo',
                'SE':'Sergipe',
                'TO':'Tocantins'}

In [173]:
capitais = {'AC':'Rio Branco',
            'AL':'Maceio',
            'AP':'Macapa',
            'AM':'Manaus',
            'BA':'Salvador',
            'CE':'Fortaleza',
            'DF':'Brasilia',
            'ES':'Vitoria',
            'GO':'Goiania',
            'MA':'Sao Luis',
            'MT':'Cuiaba',
            'MS':'Campo Grande',
            'MG':'Belo Horizonte',
            'PA':'Belem',
            'PB':'Joao Pessoa',
            'PR':'Curitiba',
            'PE':'Recife',
            'PI':'Teresina',
            'RJ':'Rio de Janeiro',
            'RN':'Natal',
            'RS':'Porto Alegre',
            'RO':'Porto Velho',
            'RR':'Boa Vista',
            'SC':'Florianopolis',
            'SP':'Sao Paulo',
            'SE':'Aracaju',
            'TO':'Palmas'}
capitais = dict((k.upper(), v.upper()) for k, v in capitais.items())

In [174]:
df['capitais'] = pd.Series(list(zip(df.sigla_unidade_federativa, df.nome_municipio))).isin(capitais.items())

In [175]:
regioes = {'Norte':['AC','AP','AM','PA','RO','RR','TO'],
           'Nordeste':['AL','BA','CE','MA','PB','PE','PI','RN','SE'],
           'Centro-Oeste':['GO','MT','MS'],
           'Sudeste':['ES','MG','RJ','SP'],
           'Sul':['PR','SC','RS']}

In [176]:
df['regiao'] = df.sigla_unidade_federativa.apply(lambda x: ''.join([k for k, v in regioes.items() if x in v]))

# Lista de população, tamanho e densidade por município

In [177]:
url = 'https://pt.wikipedia.org/wiki/Lista_de_munic%C3%ADpios_do_Brasil_por_popula%C3%A7%C3%A3o_(2020)'
municipio_pop = pd.read_html(url, thousands=' ')[0]

# eliminar espaço para converter para int
municipio_pop['População'] = municipio_pop['População'].str.replace('\D', '').astype(int)

# converter estado e municipio para sem acentos e maiusculo
municipio_pop['Unidade federativa'] = municipio_pop['Unidade federativa'].str.upper().apply(lambda x: unidecode(x))
municipio_pop['Município'] = municipio_pop['Município'].str.upper().apply(lambda x: unidecode(x))

# abreviar UF
municipio_pop['sigla_uf'] = municipio_pop['Unidade federativa'].str.upper().apply(lambda x: [k for k, v in sigla_estado.items() if (unidecode(v.upper()) == x)])
municipio_pop = municipio_pop.explode('sigla_uf')

municipio_pop.drop('Posição', axis=1, inplace=True)
municipio_pop.set_index('Código IBGE', inplace=True)

In [178]:
url = 'https://pt.m.wikipedia.org/wiki/Lista_de_munic%C3%ADpios_brasileiros_por_%C3%A1rea_decrescente'
municipio_km = pd.read_html(url, thousands=' ')[0]

municipio_km['Área (km²)'] = municipio_km['Área (km²)'].str.replace(',','.').str.replace(u'\xa0', u'').astype(float)
municipio_km.drop(['Posição','Município', 'Unidade federativa'], axis=1, inplace=True)
municipio_km.rename(columns={'Código do IBGE':'Código IBGE'}, inplace=True)
municipio_km.set_index('Código IBGE', inplace=True)

In [179]:
density_label = ['0-5','5-10','10-20','20-50','50-100','100-200','200-20k']

def agrupar_por_densidade(den):
    if den >= 0 and den < 5:
        return 0
    elif den >= 5 and den < 10:
        return 1
    elif den >= 10 and den < 20:
        return 2
    elif den >= 20 and den < 50:
        return 3
    elif den >= 50 and den < 100:
        return 4
    elif den >= 100 and den < 200:
        return 5
    elif den >= 200 and den < 20000:
        return 6
    else:
        return 'other'
    
municipios = pd.merge(municipio_pop, municipio_km, on='Código IBGE')
municipios['densidade'] = municipios['População'] / municipios['Área (km²)']
municipios['grupo_densidade'] = municipios.apply(lambda x : agrupar_por_densidade(x.densidade), axis=1)

In [180]:
# Obter tabela de correspondencia entre codigo de municipio do IBGE e TSE
link = 'https://raw.githubusercontent.com/estadao/como-votou-sua-vizinhanca/master/data/votos/correspondencia-tse-ibge.csv'
df_equivalencia = pd.read_csv(link)
df_equivalencia.set_index('COD_TSE', drop=False, inplace=True)
df_equivalencia.drop(['chave','NOME','UF'], axis=1, inplace=True)

In [181]:
df.set_index('codigo_municipio', drop=False, inplace=True)
df = df.join(df_equivalencia)

In [182]:
df.set_index('GEOCOD_IBGE', drop=False, inplace=True)
df = df.join(municipios[['População','densidade','grupo_densidade']])

In [183]:
# Obter tabela latitude e longitude para municipios
link = 'https://raw.githubusercontent.com/kelvins/Municipios-Brasileiros/main/csv/municipios.csv'
df_lat_long = pd.read_csv(link)
df_lat_long.set_index('codigo_ibge', inplace=True)
df_lat_long.drop(['nome','capital','codigo_uf'], axis=1, inplace=True)

In [184]:
df = df.join(df_lat_long)

# Candidatos por tamanho do município

In [185]:
GRANDE = 500000
PEQUENO = 100000
ELEITO = ['ELEITO','ELEITO POR MEDIA','ELEITO POR QP']

In [186]:
df_out = pd.DataFrame()
for partido in df.sigla_partido.unique():
    d = df[df.sigla_partido == partido]

    row = pd.Series({'cand_total':d.shape[0],
                     'cand_total_ele':d[(d.totalizacao_turno.isin(ELEITO))].shape[0],
                     'cand_cap_grandes':d[(d.capitais == True) & (d['População'] >= GRANDE)].shape[0],
                     'cand_cap_grandes_ele':d[(d.capitais == True) & (d['População'] >= GRANDE) & (d.totalizacao_turno.isin(ELEITO))].shape[0],
                     'cand_outras_grandes':d[(d.capitais == False) & (d['População'] >= GRANDE)].shape[0],
                     'cand_outras_grandes_ele':d[(d.capitais == False) & (d['População'] >= GRANDE) & (d.totalizacao_turno.isin(ELEITO))].shape[0],
                     'cand_cap_medias':d[(d.capitais == True) & (d['População'] >= PEQUENO) & (d['População'] < GRANDE)].shape[0],
                     'cand_cap_medias_ele':d[(d.capitais == True) & (d['População'] >= PEQUENO) & (d['População'] < GRANDE) & (d.totalizacao_turno.isin(ELEITO))].shape[0],
                     'cand_medias':d[(d.capitais == False) & (d['População'] >= PEQUENO) & (d['População'] < GRANDE)].shape[0],
                     'cand_medias_ele':d[(d.capitais == False) & (d['População'] >= PEQUENO) & (d['População'] < GRANDE) & (d.totalizacao_turno.isin(ELEITO))].shape[0],
                     'cand_pequenas':d[d['População'] < PEQUENO].shape[0],
                     'cand_pequenas_ele':d[(d['População'] < PEQUENO) & (d.totalizacao_turno.isin(ELEITO))].shape[0]},
                     name=partido)
    df_out = df_out.append(row)

In [187]:
df_out

,cand_cap_grandes,cand_cap_grandes_ele,cand_cap_medias,cand_cap_medias_ele,cand_medias,cand_medias_ele,cand_outras_grandes,cand_outras_grandes_ele,cand_pequenas,cand_pequenas_ele,cand_total,cand_total_ele
PP,620.0,31.0,55.0,4.0,3562.0,266.0,586.0,31.0,27785.0,5840.0,32608.0,6172.0
PT,636.0,41.0,64.0,2.0,3787.0,155.0,621.0,34.0,21059.0,2343.0,26167.0,2575.0
PTB,693.0,18.0,76.0,2.0,3470.0,181.0,523.0,17.0,15057.0,2157.0,19819.0,2375.0
MDB,639.0,33.0,73.0,5.0,4100.0,308.0,659.0,48.0,32472.0,6715.0,37943.0,7109.0
PODE,679.0,30.0,94.0,3.0,3615.0,190.0,556.0,16.0,12243.0,1211.0,17187.0,1450.0
DEM,696.0,39.0,68.0,3.0,3916.0,269.0,618.0,26.0,22843.0,3850.0,28141.0,4187.0
PSB,651.0,30.0,86.0,6.0,3365.0,173.0,535.0,20.0,18019.0,2667.0,22656.0,2896.0
PSD,645.0,38.0,77.0,4.0,4394.0,327.0,667.0,29.0,27836.0,5104.0,33619.0,5502.0
PSDB,652.0,34.0,94.0,5.0,3701.0,317.0,700.0,49.0,22789.0,3865.0,27936.0,4270.0
PATRI,650.0,22.0,100.0,2.0,3058.0,86.0,629.0,18.0,7395.0,565.0,11832.0,693.0


In [220]:
df_taxa = pd.DataFrame()
df_taxa['capitais'] = df_out.apply(lambda x : pd.Series(x['cand_cap_grandes_ele']+x['cand_cap_medias_ele']).div(x['cand_cap_grandes']+x['cand_cap_medias']).replace(np.inf,0).fillna(0), 1).T.squeeze()
df_taxa['outras_grandes'] = df_out.apply(lambda x : x[['cand_outras_grandes_ele']].div(x['cand_outras_grandes']).replace(np.inf,0).fillna(0), 1).T.squeeze()
df_taxa['medias'] = df_out.apply(lambda x : x[['cand_medias_ele']].div(x['cand_medias']).replace(np.inf,0).fillna(0), 1).T.squeeze()
df_taxa['pequenas'] = df_out.apply(lambda x : x[['cand_pequenas_ele']].div(x['cand_pequenas']).replace(np.inf,0).fillna(0), 1).T.squeeze()

In [221]:
df_taxa['ideologia'] = df_taxa.index.map(mapa_ideologico)

In [222]:
df_taxa.index.name = 'partido'
df_taxa.reset_index(inplace=True)
df_taxa.partido.replace('PRB','REPUBLICANOS', inplace=True)
df_taxa.partido.replace('AVT','AVANTE', inplace=True)
df_taxa.partido.replace('CDD','CIDADANIA', inplace=True)
df_taxa.partido.replace('SD','SOLIDARIEDADE', inplace=True)
df_taxa.partido.replace('PATRI','PATRIOTA', inplace=True)
df_taxa.set_index('partido', inplace=True)
df_taxa.iloc[:,:4] = df_taxa.iloc[:,:4] * 100

In [223]:
df_taxa

,capitais,outras_grandes,medias,pequenas,ideologia
partido,,,,,
PP,5.185185,5.290102,7.467715,21.018535,Centro
PT,6.142857,5.475040,4.092950,11.125884,Esquerda
PTB,2.600780,3.250478,5.216138,14.325563,Centro
MDB,5.337079,7.283763,7.512195,20.679355,Centro
PODE,4.269082,2.877698,5.255878,9.891366,Direita
DEM,5.497382,4.207120,6.869254,16.854179,Direita
PSB,4.884668,3.738318,5.141159,14.801043,Esquerda
PSD,5.817175,4.347826,7.441966,18.335968,Centro
PSDB,5.227882,7.000000,8.565253,16.959937,Centro


In [162]:
partido = 'PSOL'
fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 15,
      thickness = 20,
      line = dict(color = "black", width = 0.5),
      label = ["Candidatos", "Capitais", "Outras Grandes Grandes", "Cidades Médias", "Cidades Pequenas",
               "Eleitos"],
      color = "blue"
    ),
    link = dict(
      source = [0, 0, 0, 0, 1, 2, 3, 4],
      target = [1, 2, 3, 4, 5, 5, 5, 5],
      value = [df_out.loc[partido,'cand_cap_grandes']+df_out.loc[partido,'cand_cap_medias'],
               df_out.loc[partido,'cand_outras_grandes'],
               df_out.loc[partido,'cand_medias'],
               df_out.loc[partido,'cand_pequenas'],
               df_out.loc[partido,'cand_cap_grandes_ele']+df_out.loc[partido,'cand_cap_medias_ele'],
               df_out.loc[partido,'cand_outras_grandes_ele'],
               df_out.loc[partido,'cand_medias_ele'],
               df_out.loc[partido,'cand_pequenas_ele']]
  ))])

fig.update_layout(title_text="Basic Sankey Diagram", font_size=10)
fig.show()